<h1>Web Scraping para Sites de Novels Chinesas</h1>
<p>Este script foi desenvolvido para facilitar o acesso às melhores novels chinesas, selecionadas com base no gosto do programador que o criou.</p>
<p><strong>Funcionalidades:</strong></p>
<ul>
    <li>Exibe uma lista de novels altamente recomendadas para leitura.</li>
    <li>Permite ao usuário escolher uma novel da lista.</li>
    <li>Oferece a opção de selecionar quantos capítulos deseja ler.</li>
    <li>Permite definir o capítulo inicial para começar a leitura.</li>
    <li>Salva todos os capítulos selecionados em um arquivo <code>.txt</code>, com título e conteúdo formatados.</li>
</ul>
<p><strong>Objetivo:</strong> Tornar a leitura de novels chinesas acessível e organizada, armazenando o conteúdo de maneira prática para consulta offline.</p>


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from googletrans import Translator 

<h1>GET TOTAL PAGES</h1>
<p>Este módulo é responsável por determinar o número total de páginas disponíveis em uma novel no site.</p>
<p><strong>Funcionalidade:</strong></p>
<ul>
    <li>Acessa a URL da novel e analisa o conteúdo de cada página.</li>
    <li>Identifica quando a navegação atinge o final da lista de capítulos.</li>
    <li>Retorna o número total de páginas disponíveis para a novel.</li>
</ul>
<p><strong>Uso:</strong> Essencial para mapear todos os capítulos de uma novel, permitindo que o script acesse e colete links de capítulos de forma eficiente.</p>


In [26]:
def get_total_pages(novel_url, header):

    total_page = 1
    previous_page_content = None
    max_pages = 500

    list_page = []

    while total_page <= max_pages:
        novel_page = f"{novel_url}?page={total_page}"
        response = requests.get(novel_page, headers=header)    




        soup = BeautifulSoup(response.content, "html.parser")
        current_page_content = soup.select(".list-chapter li a")
        list_page.append(current_page_content)
        
        if previous_page_content == current_page_content:
  
            break

        previous_page_content = current_page_content
        total_page += 1

    return total_page -1 

<h1>ALL CHAPTERS ON PAGES</h1>

<p>Este módulo coleta todos os links de capítulos disponíveis para uma novel, página por página.</p>
<p><strong>Funcionalidade:</strong></p>
<ul>
    <li>Navega por cada página da novel utilizando a URL base.</li>
    <li>Identifica os links individuais de cada capítulo através de seletores HTML.</li>
    <li>Armazena todos os links em uma lista organizada.</li>
</ul>
<p><strong>Uso:</strong> Essencial para criar um mapeamento completo dos capítulos da novel, permitindo que o conteúdo seja acessado e processado posteriormente.</p>


In [ ]:
def get_all_chapter_links(novel_url, header):
    total_pages = get_total_pages(novel_url, header)
    all_chapters = []

    for page in range(1, total_pages + 1):
        page_url = f"{novel_url}?page={page}"

        response = requests.get(page_url, headers=header)
        if response.status_code != 200:
            print("Error in Script. Try again.")
            continue

        
        soup = BeautifulSoup(response.content, "html.parser")
        chapters = soup.select(".list-chapter a")
        
        for chapter in chapters:
            chapter_link = f"https://novelfull.com{chapter['href']}"
            all_chapters.append(chapter_link)
            

    return all_chapters



<h1>SCRAPING OF CHAPTER CONTENT, SUCH AS TITLE</h1>

<p>Este módulo é responsável por obter as informações completas de um capítulo específico de uma novel.</p>
<p><strong>Funcionalidade:</strong></p>
<ul>
    <li>Acessa a página de um capítulo usando o link fornecido.</li>
    <li>Extrai o título do capítulo utilizando seletores específicos do HTML.</li>
    <li>Coleta o conteúdo principal do capítulo, garantindo a formatação adequada, como quebras de linha.</li>
</ul>
<p><strong>Importância:</strong> Permite acessar e salvar o conteúdo textual detalhado de cada capítulo, essencial para leitura offline ou análise posterior.</p>
<p><strong>Uso:</strong> Pode ser integrado para criar arquivos consolidados de capítulos, como compilações completas de novels.</p>


In [ ]:
def scrape_chapter_content(chapter_url, header):
    response = requests.get(chapter_url, headers=header)
    if response.status_code != 200:
        print("ERROR")
        return None
    
    soup = BeautifulSoup(response.content, "html.parser")
    chapter_title = soup.select_one(".chapter-title").text.strip()
    chapter_content = soup.select_one(".chapter-c").get_text(separator="/n").strip()
    return chapter_title, chapter_content



<h1>SAVES THE CHAPTER IN FILE TXT</h1>



<p>Este módulo permite salvar o conteúdo dos capítulos selecionados em um único arquivo de texto.</p>
<p><strong>Funcionalidade:</strong></p>
<ul>
    <li>Recebe o conteúdo de vários capítulos, incluindo títulos e texto principal.</li>
    <li>Cria um arquivo <code>.txt</code> com o nome personalizado, indicando a novel e os capítulos incluídos.</li>
    <li>Garante a formatação adequada do texto para facilitar a leitura.</li>
</ul>
<p><strong>Importância:</strong> Oferece uma forma prática de armazenar os capítulos para leitura offline, preservando o conteúdo de forma organizada.</p>
<p><strong>Uso:</strong> Ideal para usuários que desejam acessar suas novels favoritas sem depender de conexão com a internet ou leitura direta no navegador.</p>


In [24]:
def saved_chapter(novel_url, start_chapter, num_chapters, output_file,header):

    all_chapters= get_all_chapter_links(novel_url, header)

    if start_chapter > len(all_chapters) or start_chapter <=0:
        print("Esse capitulo não existe, tente novamente.")
        return


    #seleciona os capitulos desejados.
    chapter_urls = all_chapters[start_chapter - 1: start_chapter + num_chapters -1]

        
    with open(output_file, "w", encoding="utf-8") as f:
        for chapter_url in chapter_urls:
            title, content = scrape_chapter_content(chapter_url, header)

            if title and content:
                content = content.replace("/n", "\n")
                f.write(f"{title}\n\n{content}\n\n")
                print(f"capitulo {title} salvo com sucesso")




<h1>SYNOPSIS FROM EVERY NOVEL</h1>

<p>Este módulo coleta e exibe a sinopse de cada novel disponível no site.</p>
<p><strong>Funcionalidade:</strong></p>
<ul>
    <li>Extrai o resumo ou descrição principal de cada novel usando web scraping.</li>
    <li>Fornece uma visão geral rápida da história, ajudando o usuário a escolher qual novel deseja ler.</li>
    <li>As sinopses são armazenadas em um DataFrame para fácil acesso e podem ser exibidas juntamente com outras informações.</li>
</ul>
<p><strong>Importância:</strong> Permite ao usuário entender o enredo básico de cada novel antes de fazer sua escolha.</p>
<p><strong>Uso:</strong> Ideal para leitores que procuram novas histórias ou desejam explorar diferentes gêneros.</p>




In [ ]:
#"desc-text"

def scrape_sinopse(chapter_url, header):
    response = requests.get(chapter_url, headers=header)
    if response.status_code != 200:
        print("ERROR")
        return None
    
    soup = BeautifulSoup(response.content, "html.parser")
    chapter_sinopse = soup.select_one(".desc-text").text.strip()

    return chapter_sinopse

    
    
    


<h1>RATINGS AND TOTAL WATCHED</h1>
<p>Este módulo coleta informações sobre a avaliação e a popularidade de cada novel.</p>
<p><strong>Funcionalidade:</strong></p>
<ul>
    <li>Obtém a classificação (rating) média de cada novel, refletindo a qualidade percebida pelos leitores.</li>
    <li>Coleta o número total de visualizações ou leituras, indicando a popularidade de cada história.</li>
    <li>As informações são armazenadas em um DataFrame para análise e exibição.</li>
</ul>
<p><strong>Importância:</strong></p>
<ul>
    <li>Ajuda o usuário a identificar as novels mais apreciadas e mais lidas.</li>
    <li>Fornece critérios adicionais para selecionar uma história, além da sinopse.</li>
</ul>
<p><strong>Uso:</strong> Recomendado para leitores que desejam explorar histórias bem avaliadas ou altamente populares.</p>


In [ ]:
def scrape_rating(chapter_url, header):
    #small
    response = requests.get(chapter_url, headers=header)
    if response.status_code != 200:
        print("Error")
        return None

    soup = BeautifulSoup(response.content, "html.parser")
    chapter_rating = soup.select_one(".small").text.strip()

    chapter_rating = chapter_rating.replace("\r", ":")
    return chapter_rating
    

<h1>DATA FRAME PREVIEW</h1>

<p>Este segmento do código é responsável por estruturar e exibir os dados coletados das novels em um formato de DataFrame, facilitando a análise e manipulação das informações.</p>
<p><strong>Funcionalidades principais:</strong></p>
<ul>
    <li>Criação de um DataFrame contendo as seguintes colunas:
        <ul>
            <li><strong>Título:</strong> Nome da novel.</li>
            <li><strong>URL:</strong> Link da página principal da novel.</li>
            <li><strong>Sinopse:</strong> Breve descrição da história da novel.</li>
            <li><strong>Rating:</strong> Avaliação da novel e total de visualizações (se disponível).</li>
        </ul>
    </li>
    <li>Adiciona dinamicamente as colunas <em>Sinopse</em> e <em>Rating</em> utilizando funções de web scraping aplicadas aos links fornecidos.</li>
    <li>Configura o ambiente de exibição para apresentar todos os dados de forma completa, incluindo todas as linhas, colunas e textos extensos.</li>
</ul>
<p><strong>Benefício:</strong> Com esta estrutura, o usuário pode visualizar e comparar as informações de todas as novels disponíveis em um único local.</p>



In [ ]:
header = {
    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0'
}

a = {
    "Contra os Deuses": "https://novelfull.com/against-the-gods.html" ,
    "Deus da Medicina Incomparável": "https://novelfull.com/unrivaled-medicine-god.html",
    "Deus Marcial Inigualável": "https://novelfull.com/peerless-martial-god.html",
    "A Mecânica Lendária": "https://novelfull.com/the-legendary-mechanic.html",
    "Reverendo Insanidade": "https://novelfull.com/reverend-insanity.html" ,
    "Antigo Monarca": "https://novelfull.com/ancient-godly-monarch.html",
    "Rei dos Deuse": "https://novelfull.com/king-of-gods.html",
    "Deus da Matança": "https://novelfull.com/king-of-gods.html",
    "Técnica de Fortalecimento Ancestral": "https://novelfull.com/ancient-strengthening-technique.html",
    "Verdadeiro Mundo Marcial": "https://novelfull.com/true-martial-world.html",
    "Filho Abandonado Mais Forte": "https://novelfull.com/strongest-abandoned-son.html",
    "Paraíso dos Monstros": "https://novelfull.com/monster-paradise.html",
    "Imortal Renegado": "https://novelfull.com/renegade-immortal.html",
    "Estrela engolida": "https://novelfull.com/swallowed-star.html",
    "Bruxo do Mundo dos Magos": "https://novelfull.com/warlock-of-the-magus-world.html",
    "Monarca do Mal de Outro Mundo": "https://novelfull.com/otherworldly-evil-monarch.html",
    "Retornando do Mundo Imortal": "https://novelfull.com/returning-from-the-immortal-world.html",
    "O irmão sênior mais forte da história": "https://novelfull.com/historys-strongest-senior-brother.html",
    "Wu Dong Qian Kun": "https://novelfull.com/wu-dong-qian-kun.html",
    "Deus Marcial Inigualável 2": "https://novelfull.com/peerless-martial-god-2.html",
    "Grande Rei Demônio": "https://novelfull.com/great-demon-king.html",
    "Deus e o Mundo do Diabo": "https://novelfull.com/god-and-devil-world.html",
    "Paraíso dos Deuses Demoníacos": "https://novelfull.com/paradise-of-demons-and-gods.html",

    }

data = pd.DataFrame(list(a.items()), columns=["titulo", "Url"])

#adciona a coluna de sinopse
data["Sinopse"] =data["Url"].apply(lambda url: scrape_sinopse(url, header))

#adciona a coluna de avaliações
data["Rating"] = data["Url"].apply(lambda url: scrape_rating(url, header))


# Configurar para exibir todas as linhas e colunas
pd.set_option('display.max_rows', None)  # Exibe todas as linhas
pd.set_option('display.max_columns', None)  # Exibe todas as colunas
pd.set_option('display.width', None)  # Ajusta a largura da tabela
pd.set_option('display.max_colwidth', None)  # Exibe texto completo nas células


data





<h1>INTERACTIVE USER INPUT FOR CHAPTER SELECTION</h1>
<p>Este segmento do código permite que o usuário escolha uma novel e defina os parâmetros de leitura, como o número de capítulos que deseja ler e o capítulo inicial.</p>
<p><strong>Funcionalidades principais:</strong></p>
<ul>
    <li>Solicita ao usuário a escolha da novel a partir de uma tabela pré-definida, utilizando um número de índice (<code>choise_novel</code>).</li>
    <li>Pede ao usuário para definir quantos capítulos deseja ler (<code>num_chapters</code>) e qual será o primeiro capítulo da leitura (<code>start_chapter</code>).</li>
    <li>Com base na escolha da novel, o código extrai a URL correspondente da tabela de dados (<code>novel_url</code>), permitindo o acesso ao conteúdo da novel.</li>
    <li>Cria dinamicamente um nome para o arquivo de saída (<code>output_file</code>) com base no título da novel e no intervalo de capítulos selecionados.</li>
    <li>Chama a função <code>saved_chapter</code> para iniciar o processo de scraping dos capítulos desejados e salvá-los no arquivo de saída.</li>
</ul>
<p><strong>Benefício:</strong> Essa interação com o usuário oferece uma maneira simples e personalizada de escolher uma novel, definir a quantidade de capítulos e armazená-los de forma organizada para leitura posterior.</p>


In [ ]:


choise_novel = int(input("De acordo com a tabela de novels, escolha a novel baseado no numero: "))
num_chapters = int(input("Quantos capitulos vc quer ler?  "))
start_chapter = int(input("Qual capitulo vc quer iniciar? "))
novel_url = data["Url"][choise_novel]


output_file = f"{data['titulo'][choise_novel]} do capitulo {start_chapter} até {start_chapter + num_chapters}"

b = saved_chapter(novel_url, start_chapter, num_chapters, output_file, header)